In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import re
import os
tf.__version__
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
# df = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/train.csv")
# df_val = pd.read_csv("/content/drive/MyDrive/Colab_Notebooks/val.csv")


In [3]:
# def clean_data(text):
#     text = text.lower()
#     text = re.sub(r"[-=+*\"#@!$%^&()`<>\[\]]","",text)
#     text = re.sub(r"i'm","i am",text)
#     text = re.sub(r"he's","he is",text)
#     text = re.sub(r"she's","she is",text)
#     text = re.sub(r"it's","it is",text)
#     text = re.sub(r"they're","they are",text)
#     text = re.sub(r"there're","there are",text)
#     text = re.sub(r"there's","there is",text)
#     text = re.sub(r"how're","how are",text)
#     text = re.sub(r"what're","what are",text)
#     text = re.sub(r"where're","where ",text)
#     text = re.sub(r"who're","who are",text)
#     text = re.sub(r"that're","that are",text)
#     text = re.sub(r"when're","when are",text)
#     text = re.sub(r"how's","how is",text)
#     text = re.sub(r"what's","what is",text)
#     text = re.sub(r"where's","where is",text)
#     text = re.sub(r"who's","who is",text)
#     text = re.sub(r"that's","that is",text)
#     text = re.sub(r"when's","when is",text)
#     text = re.sub(r"won't","would not",text)
#     text = re.sub(r"nt't","can not",text)
#     text = re.sub(r"\'bout'","about",text)
#     text = re.sub(r"\'till'","untill",text)
#     text = re.sub(r"\'ll","will",text)
#     text = re.sub(r"\'ve","have",text)
#     text = re.sub(r"\'re","are",text)
#     text = re.sub(r"\'d","would",text)
#     text = re.sub(r"\."," ",text)
#     text = re.sub(r"\,"," ",text)
#     text = re.sub(r"\!"," ",text)
#     text = re.sub(r"\?"," ",text)
#     text = re.sub(r"\;"," ",text)
#     text = re.sub(r"\:"," ",text)
#     return text

    
# df['text'] = df['text'].apply(clean_data)
# df['augmented_text'] = df['augmented_text'].apply(clean_data)
# df_val['text'] = df['text'].apply(clean_data)
# df_val['augmented_text'] = df['augmented_text'].apply(clean_data)



In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/Copy of data.csv')
# df = df[:82000]
train_df = df['augmented_text']
test_df = df['text']
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(train_df, test_df, test_size=0.5, random_state=24)
val_x, test_x,val_y, test_y = train_test_split(test_x, test_y, test_size = 0.8, random_state=24)


In [5]:
train_x

665124    amerkdan citizen who xiev in darwin prison mus...
614705                      ddney fc nwcastle jets a league
725648    politicinsa with unfortunate nicknames that sitkc
62804     ejff horn trainer eglnn rushton deefdns action...
458699            mandatory glucose testing teyp 1 dibaeets
                                ...                        
478609           local govt trojp backs highway revxm9 gnds
516439    png rcciket manager says team has let two gold...
211136           fitrzyo water plan  poropsal to open awtre
899                efth city bxnq smoking in pedestrian aas
242082           bowierc can be hero3z at the icc wirls cup
Name: augmented_text, Length: 461184, dtype: object

In [6]:
val_x

401835    wxunh in rughing for australian of the year xwqrd
889764       daeladie zoo apndsa emekrats get easter treats
376080    iaea offers to guarantee ifahs nucpea fhe  supply
860972    heated debate vore call for fouhrt warrnambool...
260601    cricket mtahc interrupted by rcossobw btol on ...
                                ...                        
419844            hivt sets stolen gendgations apology date
741935    rmma hcar eyects latest allegations of misconduct
238252      acremn brown speaks to tony gifrfiths boaut the
912800        uq vice chancellor brigrs eodward resignation
454594                briedg collapse in genoa kills dozens
Name: augmented_text, Length: 92237, dtype: object

In [7]:
max_length = 8

In [8]:

g_text_tokenizer = Tokenizer()
g_text_tokenizer.fit_on_texts(train_x)
g_text_word_index = g_text_tokenizer.word_index
train_sequences = g_text_tokenizer.texts_to_sequences(train_x)
train_padded = pad_sequences(train_sequences, maxlen = max_length, padding='post')


text_tokenizer = Tokenizer()
text_tokenizer.fit_on_texts(train_y)
text_word_index = text_tokenizer.word_index
test_sequences = text_tokenizer.texts_to_sequences(train_y)
test_padded = pad_sequences(test_sequences,maxlen = max_length, padding='post')





In [9]:
val_train_sequences = g_text_tokenizer.texts_to_sequences(val_x)
val_train_padded = pad_sequences(val_train_sequences, maxlen = max_length, padding='post')

val_test_sequences = text_tokenizer.texts_to_sequences(val_y)
val_test_padded = pad_sequences(val_test_sequences, maxlen = max_length, padding='post')

In [11]:
g_vocab_size = len(g_text_word_index)+1
vocab_size = len(text_word_index)+1
print(g_vocab_size, vocab_size)

688249 74657


In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=g_vocab_size, output_dim=1028, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(vocab_size,activation='softmax')
])
learning_rate = 0.01
optimizer = tf.keras.optimizers.RMSprop(learning_rate)
optimizer = tf.keras.mixed_precision.LossScaleOptimizer(optimizer)
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=optimizer,
              metrics=['accuracy'])

In [14]:
model.summary(expand_nested=True,
    show_trainable=True)

Model: "sequential"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 embedding (Embedding)       (None, 8, 1028)           70751997  Y          
                                                       2                    
                                                                            
 bidirectional (Bidirectiona  (None, 8, 512)           2631680   Y          
 l)                                                                         
                                                                            
 dropout (Dropout)           (None, 8, 512)            0         Y          
                                                                            
 bidirectional_1 (Bidirectio  (None, 8, 512)           1574912   Y          
 nal)                                                                       
                                                        

In [15]:
train_padded.shape

(461184, 8)

In [16]:
temp_test_padded = test_padded.reshape((test_padded.shape[0], test_padded.shape[1],1))
temp_train_padded = train_padded.reshape((train_padded.shape[0], train_padded.shape[1],1))


In [17]:
epoch = 20
model.fit(train_padded, test_padded, batch_size=512, epochs=epoch, validation_data = (val_train_padded, val_test_padded))

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:439: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 707519972 elements. This may consume a large amount of memory.
  num_elements)


901/901 [==============================] - 458s 498ms/step - loss: 4.6550 - accuracy: 0.4218 - val_loss: 10.3255 - val_accuracy: 0.4613
Epoch 2/20
901/901 [==============================] - 447s 496ms/step - loss: 3.4067 - accuracy: 0.5595 - val_loss: 12.9525 - val_accuracy: 0.4996
Epoch 3/20
901/901 [==============================] - 447s 496ms/step - loss: 3.0202 - accuracy: 0.5962 - val_loss: 15.2032 - val_accuracy: 0.5180
Epoch 4/20
901/901 [==============================] - 447s 496ms/step - loss: 2.7695 - accuracy: 0.6200 - val_loss: 17.2753 - val_accuracy: 0.5298
Epoch 5/20
901/901 [==============================] - 447s 496ms/step - loss: 2.5773 - accuracy: 0.6383 - val_loss: 17.0032 - val_accuracy: 0.5375
Epoch 6/20
901/901 [==============================] - 447s 496ms/step - loss: 2.4295 - accuracy: 0.6539 - val_loss: 18.9542 - val_accuracy: 0.5434
Epoch 7/20
901/901 [==============================] - 446s 496ms/step - loss: 2.3106 - accuracy: 0.6677 - val_loss: 19.2280 - val

# New Section

In [18]:
model.save('/content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_93_3bidirectional_lstm_rnn_15inputlength')

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_93_3bidirectional_lstm_rnn_15inputlength/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Colab_Notebooks/20_epoch_accuracy_93_3bidirectional_lstm_rnn_15inputlength/assets


In [19]:

test_sample = test_x.iloc[:2]
test_sample = g_text_tokenizer.texts_to_sequences(test_sample)
test_sample = pad_sequences(test_sample, maxlen=max_length, padding='post')





In [20]:
test_sample.shape

(2, 8)

In [21]:
test_x.iloc[:2]

492910    polglase jnfaz3d by inquiry levql implications
491467              ruthless bajrn expose arqenzls flaws
Name: augmented_text, dtype: object

In [22]:
test_y.iloc[:2]

492910    polglase unfazed by inquiry legal implications
491467             ruthless bayern expose arsenals flaws
Name: text, dtype: object

In [23]:
pre = model.predict(test_sample)
pre.shape

(2, 8, 74657)

In [24]:
np.argmax(pre[0],1)

array([  43,   20,  148,  370, 8886,    0,    0,    0])

In [25]:
def deTokenize(logits):
    index_to_words = {id: word for word, id in text_tokenizer.word_index.items()}
    index_to_words[0] = ''
    pre_index = np.argmax(logits, 1)
    return ' '.join(index_to_words[prediction] for prediction in pre_index)

In [26]:
print("Iutput:    ",test_x.iloc[0])
print("Actual:    ",test_y.iloc[0])
print("Predicted: ",deTokenize(pre[0]))

Iutput:     polglase jnfaz3d by inquiry levql implications
Actual:     polglase unfazed by inquiry legal implications
Predicted:  charged by inquiry legal incursion   
